<a href="https://colab.research.google.com/github/seethaladevi2024-cpu/Java-Programs/blob/main/E_Commerce_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install flask -q
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

Selecting previously unselected package cloudflared.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.9.1) ...
Setting up cloudflared (2025.9.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
from flask import Flask, render_template_string, redirect, url_for, request
import threading
import subprocess
import time
import re

app = Flask(__name__)

# Sample Products Database
products = [
    {"id": 1, "name": "Wireless Headphones", "price": 79.99, "image": "https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=300"},
    {"id": 2, "name": "Smart Watch", "price": 199.99, "image": "https://images.unsplash.com/photo-1523275335684-37898b6baf30?w=300"},
    {"id": 3, "name": "Laptop Stand", "price": 49.99, "image": "https://images.unsplash.com/photo-1527864550417-7fd91fc51a46?w=300"},
    {"id": 4, "name": "USB-C Cable", "price": 12.99, "image": "https://images.unsplash.com/photo-1583863788434-e58a36330cf0?w=300"},
    {"id": 5, "name": "Mechanical Keyboard", "price": 129.99, "image": "https://images.unsplash.com/photo-1587829741301-dc798b83add3?w=300"},
    {"id": 6, "name": "Wireless Mouse", "price": 34.99, "image": "https://images.unsplash.com/photo-1527814050087-3793815479db?w=300"},
]

# Shopping Cart
cart = {}

# Homepage Template
HOME_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>TechShop - Home</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background: #f5f5f5; }
        .navbar { background: #2c3e50; color: white; padding: 1rem 2rem; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
        .navbar h1 { font-size: 1.8rem; }
        .navbar a { color: white; text-decoration: none; background: #e74c3c; padding: 0.5rem 1.5rem; border-radius: 5px; font-weight: bold; transition: background 0.3s; }
        .navbar a:hover { background: #c0392b; }
        .container { max-width: 1200px; margin: 2rem auto; padding: 0 1rem; }
        .products { display: grid; grid-template-columns: repeat(auto-fill, minmax(280px, 1fr)); gap: 2rem; }
        .product-card { background: white; border-radius: 10px; overflow: hidden; box-shadow: 0 4px 6px rgba(0,0,0,0.1); transition: transform 0.3s, box-shadow 0.3s; }
        .product-card:hover { transform: translateY(-5px); box-shadow: 0 8px 15px rgba(0,0,0,0.2); }
        .product-card img { width: 100%; height: 250px; object-fit: cover; }
        .product-info { padding: 1.5rem; }
        .product-info h3 { font-size: 1.2rem; margin-bottom: 0.5rem; color: #2c3e50; }
        .product-info .price { font-size: 1.5rem; color: #27ae60; font-weight: bold; margin-bottom: 1rem; }
        .product-info button { width: 100%; padding: 0.75rem; background: #3498db; color: white; border: none; border-radius: 5px; font-size: 1rem; cursor: pointer; transition: background 0.3s; font-weight: bold; }
        .product-info button:hover { background: #2980b9; }
        .alert { background: #2ecc71; color: white; padding: 1rem; border-radius: 5px; margin-bottom: 1rem; text-align: center; }
    </style>
</head>
<body>
    <div class="navbar">
        <h1>🛒 TechShop</h1>
        <a href="/cart">View Cart ({{ cart_count }} items)</a>
    </div>
    <div class="container">
        {% if added %}
        <div class="alert">✓ Product added to cart!</div>
        {% endif %}
        <h2 style="margin-bottom: 1.5rem; color: #2c3e50;">Featured Products</h2>
        <div class="products">
            {% for product in products %}
            <div class="product-card">
                <img src="{{ product.image }}" alt="{{ product.name }}">
                <div class="product-info">
                    <h3>{{ product.name }}</h3>
                    <div class="price">${{ "%.2f"|format(product.price) }}</div>
                    <form action="/add_to_cart/{{ product.id }}" method="POST">
                        <button type="submit">Add to Cart</button>
                    </form>
                </div>
            </div>
            {% endfor %}
        </div>
    </div>
</body>
</html>
"""

# Cart Template
CART_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>TechShop - Shopping Cart</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background: #f5f5f5; }
        .navbar { background: #2c3e50; color: white; padding: 1rem 2rem; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
        .navbar h1 { font-size: 1.8rem; }
        .navbar a { color: white; text-decoration: none; background: #3498db; padding: 0.5rem 1.5rem; border-radius: 5px; font-weight: bold; transition: background 0.3s; }
        .navbar a:hover { background: #2980b9; }
        .container { max-width: 900px; margin: 2rem auto; padding: 0 1rem; }
        .cart-empty { background: white; padding: 3rem; border-radius: 10px; text-align: center; box-shadow: 0 4px 6px rgba(0,0,0,0.1); }
        .cart-empty h2 { color: #7f8c8d; margin-bottom: 1rem; }
        .cart-item { background: white; padding: 1.5rem; margin-bottom: 1rem; border-radius: 10px; display: flex; align-items: center; gap: 1.5rem; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
        .cart-item img { width: 100px; height: 100px; object-fit: cover; border-radius: 8px; }
        .cart-item-info { flex: 1; }
        .cart-item-info h3 { color: #2c3e50; margin-bottom: 0.5rem; }
        .cart-item-info .price { color: #27ae60; font-size: 1.2rem; font-weight: bold; }
        .cart-item-info .quantity { color: #7f8c8d; margin-top: 0.5rem; }
        .cart-total { background: #34495e; color: white; padding: 2rem; border-radius: 10px; margin-top: 2rem; }
        .cart-total h2 { margin-bottom: 1rem; }
        .cart-total .total-amount { font-size: 2rem; font-weight: bold; color: #2ecc71; margin-bottom: 1.5rem; }
        .cart-total form { display: inline; }
        .cart-total button { padding: 1rem 3rem; background: #e74c3c; color: white; border: none; border-radius: 5px; font-size: 1.1rem; cursor: pointer; font-weight: bold; transition: background 0.3s; }
        .cart-total button:hover { background: #c0392b; }
    </style>
</head>
<body>
    <div class="navbar">
        <h1>🛒 TechShop</h1>
        <a href="/">← Continue Shopping</a>
    </div>
    <div class="container">
        <h2 style="margin-bottom: 1.5rem; color: #2c3e50;">Your Shopping Cart</h2>
        {% if cart_items %}
            {% for item in cart_items %}
            <div class="cart-item">
                <img src="{{ item.image }}" alt="{{ item.name }}">
                <div class="cart-item-info">
                    <h3>{{ item.name }}</h3>
                    <div class="price">${{ "%.2f"|format(item.price) }}</div>
                    <div class="quantity">Quantity: {{ item.quantity }}</div>
                </div>
                <div class="price">${{ "%.2f"|format(item.price * item.quantity) }}</div>
            </div>
            {% endfor %}
            <div class="cart-total">
                <h2>Order Summary</h2>
                <div class="total-amount">Total: ${{ "%.2f"|format(total) }}</div>
                <form action="/checkout" method="POST">
                    <button type="submit">Place Order</button>
                </form>
            </div>
        {% else %}
            <div class="cart-empty">
                <h2>Your cart is empty</h2>
                <p style="color: #95a5a6; margin-bottom: 1rem;">Add some products to get started!</p>
                <a href="/" style="color: #3498db; text-decoration: none; font-weight: bold;">Browse Products →</a>
            </div>
        {% endif %}
    </div>
</body>
</html>
"""

# Checkout Success Template
CHECKOUT_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>TechShop - Order Confirmed</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background: #f5f5f5; display: flex; align-items: center; justify-content: center; min-height: 100vh; }
        .success-box { background: white; padding: 3rem; border-radius: 10px; text-align: center; box-shadow: 0 10px 30px rgba(0,0,0,0.1); max-width: 500px; margin: 1rem; }
        .success-box .icon { font-size: 5rem; margin-bottom: 1rem; }
        .success-box h1 { color: #27ae60; margin-bottom: 1rem; }
        .success-box p { color: #7f8c8d; margin-bottom: 2rem; font-size: 1.1rem; }
        .success-box .amount { background: #ecf0f1; padding: 1rem; border-radius: 5px; font-size: 1.5rem; font-weight: bold; color: #2c3e50; margin-bottom: 2rem; }
        .success-box a { display: inline-block; padding: 1rem 2rem; background: #3498db; color: white; text-decoration: none; border-radius: 5px; font-weight: bold; transition: background 0.3s; }
        .success-box a:hover { background: #2980b9; }
    </style>
</head>
<body>
    <div class="success-box">
        <div class="icon">✓</div>
        <h1>Order Placed Successfully!</h1>
        <p>Thank you for your purchase. Your order has been confirmed.</p>
        <div class="amount">Total: ${{ "%.2f"|format(total) }}</div>
        <a href="/">Return to Home</a>
    </div>
</body>
</html>
"""

@app.route('/')
def home():
    added = request.args.get('added', False)
    cart_count = sum(cart.values())
    return render_template_string(HOME_TEMPLATE, products=products, cart_count=cart_count, added=added)

@app.route('/add_to_cart/<int:product_id>', methods=['POST'])
def add_to_cart(product_id):
    if any(p['id'] == product_id for p in products):
        cart[product_id] = cart.get(product_id, 0) + 1
    return redirect(url_for('home', added=1))

@app.route('/cart')
def view_cart():
    cart_items = []
    total = 0
    for product_id, quantity in cart.items():
        product = next((p for p in products if p['id'] == product_id), None)
        if product:
            cart_items.append({
                'id': product['id'],
                'name': product['name'],
                'price': product['price'],
                'image': product['image'],
                'quantity': quantity
            })
            total += product['price'] * quantity
    return render_template_string(CART_TEMPLATE, cart_items=cart_items, total=total)

@app.route('/checkout', methods=['POST'])
def checkout():
    total = sum(next((p['price'] for p in products if p['id'] == pid), 0) * qty for pid, qty in cart.items())
    cart.clear()
    return render_template_string(CHECKOUT_TEMPLATE, total=total)

# Start Flask
def run_flask():
    app.run(port=5000, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

print('🚀 Starting E-Commerce App...')
print('⏳ Waiting for Flask to start...\n')
time.sleep(3)

# Start Cloudflare Tunnel
print('📱 Creating public URL...\n')
tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Extract URL from output
time.sleep(5)
url_found = False
for _ in range(10):
    line = tunnel.stderr.readline()
    if 'trycloudflare.com' in line:
        match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
        if match:
            public_url = match.group(0)
            print('=' * 70)
            print('✅ E-COMMERCE STORE IS LIVE!')
            print('=' * 70)
            print(f'\n🌐 Public URL: {public_url}')
            print(f'\n👉 Click the link above to access your store!\n')
            print('=' * 70)
            url_found = True
            break
    time.sleep(1)

if not url_found:
    print('⚠️  URL not detected yet. The tunnel is starting...')
    print('Check the output below for the trycloudflare.com URL:\n')

# Keep the cell running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print('\n\n🛑 Server stopped.')

🚀 Starting E-Commerce App...
⏳ Waiting for Flask to start...

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


📱 Creating public URL...

✅ E-COMMERCE STORE IS LIVE!

🌐 Public URL: https://ranging-developmental-machine-tuning.trycloudflare.com

👉 Click the link above to access your store!



INFO:werkzeug:127.0.0.1 - - [08/Oct/2025 15:16:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2025 15:17:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2025 15:17:08] "POST /add_to_cart/1 HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2025 15:17:09] "GET /?added=1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2025 15:17:12] "GET /cart HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2025 15:17:16] "POST /checkout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2025 15:17:20] "GET / HTTP/1.1" 200 -
